In [297]:
import pandas
import plotly
import os
import plotly.express as px
import numpy as np


# Set global parameters

## TFEA differential TF motifs

In [298]:
TFEA_base_dir = "/scratch/Shares/dowell/joecardiello/FilesForMary/Heatshock/TFEA_OutputAnalysis/"
TFEAcutoffVal=0.00000001
#TFEAcutoffVal=0.000000000000000001
#TFEA_pvalcolumn="P-adj"
#TFEA_scorecolumn="E-Score"
TFEA_pvalcolumn="Corrected P-adj"
TFEA_scorecolumn="Corrected E-Score"


## other

In [256]:
marktheseTFs ={"HSF1_HUMAN.H11MO.0.A":"HSF1", "HXB2_HUMAN.H11MO.0.D":"HXB2", "HSF2_HUMAN.H11MO.0.A":"HSF2", "HSF4_HUMAN.H11MO.0.D":"HSF4"}


# Gather files we need

## label data files with a easy to use code

In [257]:
def findallTFEAresultsfiles(TFEA_base_dir):
    resultstxt = {}
    for dirpath, dirnames, filename in os.walk(TFEA_base_dir):
        for onefile in filename:
            if onefile=="results.txt":
                dirlist = dirpath.split("/")
                rootname = "__".join(dirlist[8:])
                resultstxt[rootname] = dirpath+"/"+onefile
    for key in sorted(resultstxt.keys()):
        print (key)
    return resultstxt


In [258]:
resultstxt = findallTFEAresultsfiles(TFEA_base_dir)

ATACseq__MumergeCombinedFileInput__D21_37vs42
ATACseq__MumergeCombinedFileInput__SingleSampleComparisons__D21_37
ATACseq__MumergeCombinedFileInput__SingleSampleComparisons__D21_42
ATACseq__MumergeCombinedFileInput__SingleSampleComparisons__T21_37
ATACseq__MumergeCombinedFileInput__SingleSampleComparisons__T21_42
ATACseq__MumergeCombinedFileInput__T21_37vs42
ATACseq__MumergeCombinedFileInput__t37_D21vsT21_NoEBV_M_21
ATACseq__MumergeCombinedFileInput__t42_D21vsT21_NoEBV_M_21
ATACseq__MumergeCombinedFileInput_redo__D21_37vs42
ATACseq__MumergeCombinedFileInput_redo__SingleSampleComparisons__D21_37
ATACseq__MumergeCombinedFileInput_redo__SingleSampleComparisons__D21_42
ATACseq__MumergeCombinedFileInput_redo__SingleSampleComparisons__T21_37
ATACseq__MumergeCombinedFileInput_redo__SingleSampleComparisons__T21_42
ATACseq__MumergeCombinedFileInput_redo__T21_37vs42
ATACseq__MumergeCombinedFileInput_redo__t37_D21vsT21_NoEBV_M_21
ATACseq__MumergeCombinedFileInput_redo__t42_D21vsT21_NoEBV_M_21
ATAC

## Add information about each sample to the file dataframe

In [259]:
dict_fn = {"samplename":["ATACseq__MumergeCombinedFileInput_redo__D21_37vs42", "ATACseq__MumergeCombinedFileInput_redo__T21_37vs42","PROseq__TFIT2017runs__D21_37vs42", "PROseq__TFIT2017runs__T21_37vs42", "ATACseq__MumergeCombinedFileInput_redo__t37_D21vsT21_NoEBV_M_21", "ATACseq__MumergeCombinedFileInput_redo__t42_D21vsT21_NoEBV_M_21", "PROseq__TFIT2017runs__NoChr21__t37_D21vsT21", "PROseq__TFIT2017runs__NoChr21__t42_D21vsT21"],"datatype":["ATAC", "ATAC", "PRO", "PRO","ATAC", "ATAC", "PRO", "PRO"],"sample1":["D21", "T21","D21", "T21", "D21", "D21","D21", "D21"],"sample1temp":[37,37, 37, 37, 37, 42, 37, 42],"sample2":["D21", "T21","D21", "T21", "T21", "T21","T21", "T21"],"sample2temp":[42,42, 42, 42, 37, 42, 37, 42]}


filedf = pandas.DataFrame.from_dict(dict_fn)
filedf["filename"]=filedf["samplename"]
filedf = filedf.replace({"filename": resultstxt})
filedf["shortname"] = filedf["datatype"]+"_"+filedf["sample1"]+filedf["sample1temp"].astype("str")+"_"+filedf["sample2"]+filedf["sample2temp"].astype("str")
filedf["same_person"] = filedf["sample1"]==filedf["sample2"]

filedf

,samplename,datatype,sample1,sample1temp,sample2,sample2temp,filename,shortname,same_person
0,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True
1,ATACseq__MumergeCombinedFileInput_redo__T21_37...,ATAC,T21,37,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_T2137_T2142,True
2,PROseq__TFIT2017runs__D21_37vs42,PRO,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_D2137_D2142,True
3,PROseq__TFIT2017runs__T21_37vs42,PRO,T21,37,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_T2137_T2142,True
4,ATACseq__MumergeCombinedFileInput_redo__t37_D2...,ATAC,D21,37,T21,37,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_T2137,False
5,ATACseq__MumergeCombinedFileInput_redo__t42_D2...,ATAC,D21,42,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2142_T2142,False
6,PROseq__TFIT2017runs__NoChr21__t37_D21vsT21,PRO,D21,37,T21,37,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_D2137_T2137,False
7,PROseq__TFIT2017runs__NoChr21__t42_D21vsT21,PRO,D21,42,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_D2142_T2142,False


In [260]:
def readresultstable(shortsamplename, filedf):
    onesampledf = filedf[filedf["shortname"]==shortsamplename]
    print(onesampledf["shortname"])
    filename=onesampledf["filename"].to_list()[0]
    df = pandas.read_csv(filename, sep="\t", index_col=False)
    return df


def read_with_sigcol(shortsamplename):
    df = readresultstable(shortsamplename, filedf=filedf)
    print (df.columns)
    df["cutoff"]= df.apply(lambda row : abovecuttoff(row), axis = 1)
    return df

# Ploting with one sample

## plot bar graph of pvalues


In [261]:
data = readresultstable(shortsamplename="ATAC_D2137_D2142", filedf=filedf)
data
    
    

0    ATAC_D2137_D2142
Name: shortname, dtype: object


,#TF,E-Score,Corrected E-Score,Events,GC,FPKM,P-adj,Corrected P-adj
0,ZN770_HUMAN.H11MO.0.C,-0.005870,0.041875,74885,0.664193,NaN,1.000000e+00,1.000000e-68
1,ZN770_HUMAN.H11MO.1.C,-0.011091,0.045214,59268,0.705425,NaN,1.000000e-02,1.000000e-54
2,ZN143_HUMAN.H11MO.0.A,-0.206224,-0.187719,4265,0.523364,NaN,1.000000e-43,1.000000e-35
3,THA11_HUMAN.H11MO.0.B,-0.217121,-0.198898,3809,0.522006,NaN,1.000000e-38,1.000000e-32
4,HSF1_HUMAN.H11MO.0.A,0.239771,0.239279,2653,0.431867,NaN,1.000000e-31,1.000000e-31
...,...,...,...,...,...,...,...,...
766,DPRX_HUMAN.H11MO.0.D,-0.162101,-0.158547,48,0.451353,NaN,1.000000e+00,1.000000e+00
767,HSFY1_HUMAN.H11MO.0.D,-0.150202,-0.165190,23,0.362048,NaN,1.000000e+00,1.000000e+00
768,E2F5_HUMAN.H11MO.0.B,-0.222916,-0.183009,72,0.626446,NaN,1.000000e+00,1.000000e+00
769,ZBT49_HUMAN.H11MO.0.D,-0.175912,-0.196140,71,0.336811,NaN,1.000000e+00,1.000000e+00


In [262]:
fig = px.histogram(data, x=TFEA_pvalcolumn)
fig.show()

## plot the MA plot

In [263]:
def abovecuttoff(row):
    pval= row[TFEA_pvalcolumn]
    if pval<=TFEAcutoffVal:
        col="Sig"
    else:
        col="N.S."
    return col

def MAplotresults(shortsamplename):
    df = readresultstable(shortsamplename, filedf=filedf)
    print (df.columns)
    df["cutoff"]= df.apply(lambda row : abovecuttoff(row), axis = 1)
    fig = px.scatter(df, y=df[TFEA_scorecolumn], x=np.log10(df['Events']), hover_data=['#TF', 'P-adj', 'Corrected P-adj'], color=df["cutoff"], title=shortsamplename,color_discrete_map={"Sig":"red", "N.S.":"#D3D3D3"})
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
    fig.show()
    ressig = df[df["cutoff"]=="Sig"]
    ressig = ressig.sort_values(by='P-adj', ascending=False)
    return ressig

In [264]:
s1df = MAplotresults(shortsamplename="ATAC_D2137_D2142")
s2df = MAplotresults(shortsamplename="ATAC_T2137_T2142")


0    ATAC_D2137_D2142
Name: shortname, dtype: object
Index(['#TF', 'E-Score', 'Corrected E-Score', 'Events', 'GC', 'FPKM', 'P-adj',
       'Corrected P-adj'],
      dtype='object')


/Users/allenma/jhub_venv/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning:

divide by zero encountered in log10



1    ATAC_T2137_T2142
Name: shortname, dtype: object
Index(['#TF', 'E-Score', 'Corrected E-Score', 'Events', 'GC', 'FPKM', 'P-adj',
       'Corrected P-adj'],
      dtype='object')


# The problem with a making a list of things that are significant in both

## Merge the toe data frams left to right and figure out if they are sig in one or both

In [265]:
def sig_compare(row):
    if row["cutoff_"+comparison1]=="Sig" and row["cutoff_"+comparison2]=="Sig":
        col="both"
    elif row["cutoff_"+comparison1]=="Sig" and row["cutoff_"+comparison2]!="Sig":
        col=comparison1+"_only"
    elif row["cutoff_"+comparison1]!="Sig" and row["cutoff_"+comparison2]=="Sig": 
        col=comparison2+"_only"
    else:
        col="N.S."
    return col

In [266]:
comparison1="ATAC_D2137_D2142"
comparison2="ATAC_T2137_T2142"


s1df = read_with_sigcol(shortsamplename=comparison1)
s2df = read_with_sigcol(shortsamplename=comparison2)


bothdigdf = s1df.merge(s2df, on="#TF", suffixes=("_"+comparison1, "_"+comparison2))
bothdigdf["sig_in"]= bothdigdf.apply(lambda row : sig_compare(row), axis = 1)
bothdigdf

0    ATAC_D2137_D2142
Name: shortname, dtype: object
Index(['#TF', 'E-Score', 'Corrected E-Score', 'Events', 'GC', 'FPKM', 'P-adj',
       'Corrected P-adj'],
      dtype='object')
1    ATAC_T2137_T2142
Name: shortname, dtype: object
Index(['#TF', 'E-Score', 'Corrected E-Score', 'Events', 'GC', 'FPKM', 'P-adj',
       'Corrected P-adj'],
      dtype='object')


,#TF,E-Score_ATAC_D2137_D2142,Corrected E-Score_ATAC_D2137_D2142,Events_ATAC_D2137_D2142,GC_ATAC_D2137_D2142,FPKM_ATAC_D2137_D2142,P-adj_ATAC_D2137_D2142,Corrected P-adj_ATAC_D2137_D2142,cutoff_ATAC_D2137_D2142,E-Score_ATAC_T2137_T2142,Corrected E-Score_ATAC_T2137_T2142,Events_ATAC_T2137_T2142,GC_ATAC_T2137_T2142,FPKM_ATAC_T2137_T2142,P-adj_ATAC_T2137_T2142,Corrected P-adj_ATAC_T2137_T2142,cutoff_ATAC_T2137_T2142,sig_in
0,ZN770_HUMAN.H11MO.0.C,-0.005870,0.041875,74885,0.664193,NaN,1.000000e+00,1.000000e-68,Sig,-0.009109,0.079368,74883,0.664193,NaN,1.000000e-02,1.000000e-254,Sig,both
1,ZN770_HUMAN.H11MO.1.C,-0.011091,0.045214,59268,0.705425,NaN,1.000000e-02,1.000000e-54,Sig,-0.005990,0.099478,59266,0.705425,NaN,8.070000e-01,1.000000e-257,Sig,both
2,ZN143_HUMAN.H11MO.0.A,-0.206224,-0.187719,4265,0.523364,NaN,1.000000e-43,1.000000e-35,Sig,-0.240842,-0.210398,4265,0.523364,NaN,1.000000e-51,1.000000e-38,Sig,both
3,THA11_HUMAN.H11MO.0.B,-0.217121,-0.198898,3809,0.522006,NaN,1.000000e-38,1.000000e-32,Sig,-0.254262,-0.224377,3809,0.522006,NaN,1.000000e-54,1.000000e-42,Sig,both
4,HSF1_HUMAN.H11MO.0.A,0.239771,0.239279,2653,0.431867,NaN,1.000000e-31,1.000000e-31,Sig,0.291448,0.284188,2653,0.431867,NaN,1.000000e-49,1.000000e-46,Sig,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,DPRX_HUMAN.H11MO.0.D,-0.162101,-0.158547,48,0.451353,NaN,1.000000e+00,1.000000e+00,N.S.,-0.052113,-0.051343,48,0.451353,NaN,1.000000e+00,1.000000e+00,N.S.,N.S.
767,HSFY1_HUMAN.H11MO.0.D,-0.150202,-0.165190,23,0.362048,NaN,1.000000e+00,1.000000e+00,N.S.,-0.239169,-0.275200,23,0.362048,NaN,1.000000e+00,1.000000e+00,N.S.,N.S.
768,E2F5_HUMAN.H11MO.0.B,-0.222916,-0.183009,72,0.626446,NaN,1.000000e+00,1.000000e+00,N.S.,-0.270489,-0.197567,72,0.626446,NaN,5.160000e-01,1.000000e+00,N.S.,N.S.
769,ZBT49_HUMAN.H11MO.0.D,-0.175912,-0.196140,71,0.336811,NaN,1.000000e+00,1.000000e+00,N.S.,-0.251006,-0.297437,71,0.336811,NaN,7.930000e-01,1.000000e+00,N.S.,N.S.


## scatter plot of E scores in both color by which samples its significant in

In [267]:
fig = px.scatter(bothdigdf, x =TFEA_scorecolumn+"_"+comparison1, y=TFEA_scorecolumn+"_"+comparison2, color="sig_in")
fig.show()


## scatter plot of E scores in both color by which samples its significant in with the awesome color Map!

In [268]:
fig = px.scatter(bothdigdf,x =TFEA_scorecolumn+"_"+comparison1, y=TFEA_scorecolumn+"_"+comparison2, color="sig_in", color_discrete_map={"both":"red", "N.S.":"lightgrey", comparison1:"green", comparison2:"purple"}, hover_data=["#TF"])
fig.show()


# plot set of samples

## gather data from all the files in the filedf (if you have a lot of files most of the time a long data table format is more useful)

In [326]:
def alldatafromfilesinfiledf():
    dfs = []
    for filename in filedf["filename"].to_list():
        df = pandas.read_csv(filename, sep="\t", index_col=False)
        df["filename"]= filename
        dfs.append(df)
    allsamplesTFEA = pandas.concat(dfs)
    allTFsallcomparisonsdf = filedf.merge(allsamplesTFEA, on="filename")
    return allTFsallcomparisonsdf



def extracols(allTFsallcomparisonsdf):
    allTFsallcomparisonsdf["cuttoff"] = allTFsallcomparisonsdf.apply(lambda row : abovecuttoff(row), axis = 1)
    allTFsallcomparisonsdf[["TF", "org"]]=allTFsallcomparisonsdf["#TF"].str.split("_", n=1, expand=True)
    allTFsallcomparisonsdf[TFEA_pvalcolumn+"_transform"] = allTFsallcomparisonsdf.apply(lambda row : sizetrans(row), axis = 1)
    return allTFsallcomparisonsdf


def sizetrans(row):
    #larger the shape the larder the p-value?
    s = row[TFEA_pvalcolumn]
    #s2 = 1+(s*10)
    if s!=0:
        s2 = 100+(-1*np.log(s))
    else:
        s2=100+(-1*np.log(minnot0))
    return s2



In [327]:
def defingeminnot0(allTFsallcomparisonsdf):
    nozeroallTFsallcomparisonsdf = allTFsallcomparisonsdf[allTFsallcomparisonsdf[TFEA_pvalcolumn]!=0]
    minnot0 = nozeroallTFsallcomparisonsdf[TFEA_pvalcolumn].min()
    return minnot0



allTFsallcomparisonsdf = alldatafromfilesinfiledf()
minnot0 = defingeminnot0(allTFsallcomparisonsdf)
allTFsallcomparisonsdf = extracols(allTFsallcomparisonsdf)







In [330]:
allTFsallcomparisonsdf

,samplename,datatype,sample1,sample1temp,sample2,sample2temp,filename,shortname,same_person,#TF,...,Corrected E-Score,Events,GC,FPKM,P-adj,Corrected P-adj,cuttoff,TF,org,Corrected P-adj_transform
0,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,ZN770_HUMAN.H11MO.0.C,...,0.041875,74885,0.664193,NaN,1.000000e+00,1.000000e-68,Sig,ZN770,HUMAN.H11MO.0.C,256.575786
1,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,ZN770_HUMAN.H11MO.1.C,...,0.045214,59268,0.705425,NaN,1.000000e-02,1.000000e-54,Sig,ZN770,HUMAN.H11MO.1.C,224.339595
2,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,ZN143_HUMAN.H11MO.0.A,...,-0.187719,4265,0.523364,NaN,1.000000e-43,1.000000e-35,Sig,ZN143,HUMAN.H11MO.0.A,180.590478
3,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,THA11_HUMAN.H11MO.0.B,...,-0.198898,3809,0.522006,NaN,1.000000e-38,1.000000e-32,Sig,THA11,HUMAN.H11MO.0.B,173.682723
4,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,HSF1_HUMAN.H11MO.0.A,...,0.239279,2653,0.431867,NaN,1.000000e-31,1.000000e-31,Sig,HSF1,HUMAN.H11MO.0.A,171.380138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6163,PROseq__TFIT2017runs__NoChr21__t42_D21vsT21,PRO,D21,42,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_D2142_T2142,False,CUX1_HUMAN.H11MO.0.C,...,-0.079957,139,0.474099,NaN,1.000000e+00,1.000000e+00,N.S.,CUX1,HUMAN.H11MO.0.C,100.000000
6164,PROseq__TFIT2017runs__NoChr21__t42_D21vsT21,PRO,D21,42,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_D2142_T2142,False,GSC_HUMAN.H11MO.0.D,...,-0.083933,171,0.409677,NaN,1.000000e+00,1.000000e+00,N.S.,GSC,HUMAN.H11MO.0.D,100.000000
6165,PROseq__TFIT2017runs__NoChr21__t42_D21vsT21,PRO,D21,42,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_D2142_T2142,False,PAX3_HUMAN.H11MO.0.D,...,-0.120750,137,0.213852,NaN,1.000000e+00,1.000000e+00,N.S.,PAX3,HUMAN.H11MO.0.D,100.000000
6166,PROseq__TFIT2017runs__NoChr21__t42_D21vsT21,PRO,D21,42,T21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,PRO_D2142_T2142,False,E2F5_HUMAN.H11MO.0.B,...,-0.130934,35,0.626446,NaN,1.000000e+00,1.000000e+00,N.S.,E2F5,HUMAN.H11MO.0.B,100.000000


In [331]:
allTFsallcomparisonsdf.head()

,samplename,datatype,sample1,sample1temp,sample2,sample2temp,filename,shortname,same_person,#TF,...,Corrected E-Score,Events,GC,FPKM,P-adj,Corrected P-adj,cuttoff,TF,org,Corrected P-adj_transform
0,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,ZN770_HUMAN.H11MO.0.C,...,0.041875,74885,0.664193,NaN,1.000000e+00,1.000000e-68,Sig,ZN770,HUMAN.H11MO.0.C,256.575786
1,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,ZN770_HUMAN.H11MO.1.C,...,0.045214,59268,0.705425,NaN,1.000000e-02,1.000000e-54,Sig,ZN770,HUMAN.H11MO.1.C,224.339595
2,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,ZN143_HUMAN.H11MO.0.A,...,-0.187719,4265,0.523364,NaN,1.000000e-43,1.000000e-35,Sig,ZN143,HUMAN.H11MO.0.A,180.590478
3,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,THA11_HUMAN.H11MO.0.B,...,-0.198898,3809,0.522006,NaN,1.000000e-38,1.000000e-32,Sig,THA11,HUMAN.H11MO.0.B,173.682723
4,ATACseq__MumergeCombinedFileInput_redo__D21_37...,ATAC,D21,37,D21,42,/scratch/Shares/dowell/joecardiello/FilesForMa...,ATAC_D2137_D2142,True,HSF1_HUMAN.H11MO.0.A,...,0.239279,2653,0.431867,NaN,1.000000e-31,1.000000e-31,Sig,HSF1,HUMAN.H11MO.0.A,171.380138


In [332]:
allTFsallcomparisonsdf.describe()

,sample1temp,sample2temp,E-Score,Corrected E-Score,Events,GC,FPKM,P-adj,Corrected P-adj,Corrected P-adj_transform
count,6168.000000,6168.000000,6168.000000,6.168000e+03,6168.000000,6168.000000,0.0,6.168000e+03,6168.000000,6168.000000
mean,38.250000,40.750000,0.006640,-4.463932e-18,3786.337873,0.471478,NaN,8.083409e-01,0.868483,103.853672
std,2.165239,2.165239,0.058595,4.878182e-02,8596.684598,0.154449,NaN,3.758167e-01,0.320606,26.964998
min,37.000000,37.000000,-0.418199,-3.740419e-01,0.000000,0.088889,NaN,1.000000e-131,0.000000,100.000000
25%,37.000000,40.750000,-0.013064,-1.682672e-02,461.000000,0.351827,NaN,1.000000e+00,1.000000,100.000000
50%,37.000000,42.000000,0.000816,0.000000e+00,1222.500000,0.475545,NaN,1.000000e+00,1.000000,100.000000
75%,38.250000,42.000000,0.031180,1.906987e-02,2702.000000,0.583510,NaN,1.000000e+00,1.000000,100.000000
max,42.000000,42.000000,0.430840,3.639438e-01,92895.000000,0.853449,NaN,1.000000e+00,1.000000,747.026411


In [333]:
allTFsallcomparisonsdf["shortname"].unique()

array(['ATAC_D2137_D2142', 'ATAC_T2137_T2142', 'PRO_D2137_D2142',
       'PRO_T2137_T2142', 'ATAC_D2137_T2137', 'ATAC_D2142_T2142',
       'PRO_D2137_T2137', 'PRO_D2142_T2142'], dtype=object)

In [334]:
#category_orders = {}
category_orders={"shortname": ['ATAC_D2137_D2142','ATAC_T2137_T2142',  'PRO_D2137_D2142',
       'PRO_T2137_T2142', 'ATAC_D2137_T2137', 'ATAC_D2142_T2142',
       'PRO_D2137_T2137', 'PRO_D2142_T2142']}

## plot of multiple comparisons escores and p-values

In [335]:


def gatherallsigTFslist(allTFsallcomparisonsdf, comparelist):
    subset_allTFsallcomparisonsdf = allTFsallcomparisonsdf[allTFsallcomparisonsdf["shortname"].isin(comparelist)]   
    sigTFinonedf = subset_allTFsallcomparisonsdf[subset_allTFsallcomparisonsdf["cuttoff"]=="Sig"]
    sigTFinatleastone = sigTFinonedf["#TF"].unique()
    return sigTFinatleastone
    

def filterdata_on_comp_and_TFs(allTFsallcomparisonsdf, comparelist,TFlist):
    subset_allTFsallcomparisonsdf = allTFsallcomparisonsdf[allTFsallcomparisonsdf["shortname"].isin(comparelist)] 
    print ()
    subset_sigTFsallcomparisonsdf = subset_allTFsallcomparisonsdf[subset_allTFsallcomparisonsdf["TF"].isin(TFlist)]
    return subset_sigTFsallcomparisonsdf
    

    
def plotscoreandpvalcomparison(subset_sigTFsallcomparisonsdf, height, width,category_orders):  
    fig = px.scatter(subset_sigTFsallcomparisonsdf, 
                     x="shortname", y="TF", 
                     color=TFEA_scorecolumn,
                     color_continuous_scale=px.colors.diverging.balance, 
                     symbol="cuttoff",
                     symbol_map={"Sig":"diamond", "N.S.":"circle"},
                     size=TFEA_pvalcolumn+"_transform",  
                     color_continuous_midpoint=0, category_orders=category_orders)
    fig.update_layout(height=height, width=width, plot_bgcolor='rgba(0,0,0,0)')
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_coloraxes(colorbar_len=.9)
    fig.update_layout(font=dict(
        size=18))
    fig.show()

In [336]:

def plotthetwoIwant():
    height=1500
    width=500
    comparelist= [comparison1,comparison2]
    TFlist = gatherallsigTFslist(allTFsallcomparisonsdf, comparelist)
    subset_sigTFsallcomparisonsdf = filterdata_on_comp_on_TFs(comparelist=comparelist,TFlist=TFlist)
    plotscoreandpvalcomparison(subset_sigTFsallcomparisonsdf, height=height,width=width, category_orders=category_orders)

In [337]:
plotthetwoIwant()

## plot of multiple comparisons escores and p-values sort by the number of comaprisons things are sig in

In [338]:
def sorterofscoreandpvalcomparison(subset_allTFsallcomparisonsdf):
    grouped_multiple = subset_allTFsallcomparisonsdf.groupby(['#TF']).agg({TFEA_pvalcolumn: ['mean', 'min', 'max',], "cuttoff":['sum'], TFEA_scorecolumn: ['mean', 'min', 'max', "sum"]})
    grouped_multiple.columns = [TFEA_pvalcolumn+'_mean', TFEA_pvalcolumn+'_min', TFEA_pvalcolumn+'_max', "sig_count", TFEA_scorecolumn+"_mean", TFEA_scorecolumn+"_min", TFEA_scorecolumn+"_max", TFEA_scorecolumn+"_sum"]
    grouped_multiple = grouped_multiple.reset_index()
    grouped_multiple = grouped_multiple.sort_values(["sig_count", TFEA_scorecolumn+'_sum'], ascending=[True, False])
    sorter = grouped_multiple["#TF"].tolist()
    sorterIndex = dict(zip(sorter, range(len(sorter))))
    subset_allTFsallcomparisonsdf['TF_Rank'] = subset_allTFsallcomparisonsdf['#TF'].map(sorterIndex)
    subset_allTFsallcomparisonsdf = subset_allTFsallcomparisonsdf.sort_values(["TF_Rank"], ascending=[False])
    return subset_allTFsallcomparisonsdf


def plotsortedscoreandpvalcomparison(subset_sigTFsallcomparisonsdf, height,width,category_orders):
    subset_sigTFsallcomparisonsdf = sorterofscoreandpvalcomparison(subset_sigTFsallcomparisonsdf)
    print (subset_sigTFsallcomparisonsdf.index)
    fig = px.scatter(subset_sigTFsallcomparisonsdf, 
                     x="shortname", y="TF", 
                     color=TFEA_scorecolumn,
                     color_continuous_scale=px.colors.diverging.PiYG, 
                     symbol="cuttoff",
                     symbol_map={"Sig":"diamond", "N.S.":"circle"},
                     size=TFEA_pvalcolumn+"_transform",  
                     color_continuous_midpoint=0,
                     category_orders=category_orders)
    fig.update_layout(height=height, width=width, plot_bgcolor='rgba(23,23,23,0.5)')
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_coloraxes(colorbar_len=.9)
    fig.update_layout(font=dict(
        size=18))
    fig.show()   
    
    
def plotsortedscoreandpvalcomparison_nosize(subset_sigTFsallcomparisonsdf, height,width,category_orders):
    subset_sigTFsallcomparisonsdf = sorterofscoreandpvalcomparison(subset_sigTFsallcomparisonsdf)
    print (subset_sigTFsallcomparisonsdf.index)
    fig = px.scatter(subset_sigTFsallcomparisonsdf, 
                     x="shortname", y="TF", 
                     color=TFEA_scorecolumn,
                     color_continuous_scale=px.colors.diverging.PiYG, 
                     symbol="cuttoff",
                     symbol_map={"Sig":"diamond", "N.S.":"circle"},  
                     color_continuous_midpoint=0,
                     category_orders=category_orders, hover_data=["shortname", TFEA_scorecolumn, "cuttoff", TFEA_pvalcolumn])
    fig.update_layout(height=height, width=width, plot_bgcolor='rgba(23,23,23,0.5)')
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_coloraxes(colorbar_len=.9)
    fig.update_layout(font=dict(
        size=18))
    fig.show()    

In [339]:

def plotthetwoIwant_sorted():
    height=1500
    width=500
    comparelist= [comparison1,comparison2]
    TFlist = gatherallsigTFslist(allTFsallcomparisonsdf, comparelist)
    subset_sigTFsallcomparisonsdf = filterdata_on_comp_on_TFs(comparelist=comparelist,TFlist=TFlist)
    plotsortedscoreandpvalcomparison(subset_sigTFsallcomparisonsdf, height=height,width=width, category_orders=category_orders)
    
    
    

In [340]:
plotthetwoIwant_sorted()

Int64Index([  24,  795,  777,    3,    2,  778,   18,  794,    7,  786,  782,
              11,  776,   14,  779,   25,  790,   20,   28,  801,   17,  792,
             787,    6,  774,   16,  773,    5,    0,  772,   26,  800,    1,
             771,   19,  784,  785,    9,   29,  799,  775,    4,   12,  780,
             781,    8,  783,   13,   33,  806,   30,  826,  805,   21,   34,
            1178, 1147,   35,  841,   15,   32,  815,   27,  852,   10,  809,
             811,   22,  818,   23,   31,  817,  802,   58,  791,   89,  796,
             332,  804,   68,  797,  273,  803,   70,   60,  798,   37,  789,
              55,  793,   59,  788],
           dtype='int64')


In [341]:
def plotallcomparisonsthatareHS():
    height=3000
    width=500
    comparelist= allTFsallcomparisonsdf[allTFsallcomparisonsdf["same_person"]==True]["shortname"].unique()
    TFlist = gatherallsigTFslist(allTFsallcomparisonsdf, comparelist)
    subset_sigTFsallcomparisonsdf = filterdata_on_comp_on_TFs(comparelist=comparelist,TFlist=TFlist)
    plotsortedscoreandpvalcomparison(subset_sigTFsallcomparisonsdf, height=height,width=width, category_orders=category_orders)
    

plotallcomparisonsthatareHS()

Int64Index([  16, 2315,  774, 1544,  771, 2314, 1543,    1,  801, 2323,
            ...
            1648,   50,   41, 1650,  833, 2385,  365, 2383, 1619,  823],
           dtype='int64', length=412)


In [342]:
def plotallcomparisonsthatare_D21vsT21():
    height=1500
    width=500
    comparelist= allTFsallcomparisonsdf[allTFsallcomparisonsdf["same_person"]!=True]["shortname"].unique()
    TFlist = gatherallsigTFslist(allTFsallcomparisonsdf, comparelist)
    print(TFlist)
    subset_sigTFsallcomparisonsdf = filterdata_on_comp_on_TFs(comparelist=comparelist,TFlist=TFlist)
    plotsortedscoreandpvalcomparison(subset_sigTFsallcomparisonsdf, height=height,width=width, category_orders=category_orders)
    

plotallcomparisonsthatare_D21vsT21()

['IRF4_HUMAN.H11MO.0.A' 'IRF8_HUMAN.H11MO.0.B' 'ZN121_HUMAN.H11MO.0.C'
 'TF65_HUMAN.H11MO.0.A' 'BC11A_HUMAN.H11MO.0.A' 'SP2_HUMAN.H11MO.0.A'
 'SPIB_HUMAN.H11MO.0.A' 'PAX5_HUMAN.H11MO.0.A' 'THAP1_HUMAN.H11MO.0.C'
 'SPI1_HUMAN.H11MO.0.A' 'IRF1_HUMAN.H11MO.0.A' 'SP1_HUMAN.H11MO.0.A'
 'THA_HUMAN.H11MO.0.C' 'ZN770_HUMAN.H11MO.1.C' 'ZN770_HUMAN.H11MO.0.C'
 'RELB_HUMAN.H11MO.0.C' 'SP1_HUMAN.H11MO.1.A' 'IRF3_HUMAN.H11MO.0.B'
 'STAT1_HUMAN.H11MO.1.A' 'IRF2_HUMAN.H11MO.0.A' 'RARA_HUMAN.H11MO.0.A'
 'SP3_HUMAN.H11MO.0.B' 'ZEP1_HUMAN.H11MO.0.D' 'ZN768_HUMAN.H11MO.0.C'
 'STAT2_HUMAN.H11MO.0.A' 'NFKB2_HUMAN.H11MO.0.B' 'NRF1_HUMAN.H11MO.0.A'
 'SP2_HUMAN.H11MO.1.B' 'REL_HUMAN.H11MO.0.B' 'EGR4_HUMAN.H11MO.0.D'
 'CTCFL_HUMAN.H11MO.0.A' 'TAF1_HUMAN.H11MO.0.A' 'KLF12_HUMAN.H11MO.0.C'
 'SMAD3_HUMAN.H11MO.0.B' 'TBX2_HUMAN.H11MO.0.D' 'SP4_HUMAN.H11MO.0.A'
 'THB_HUMAN.H11MO.0.C' 'PRDM1_HUMAN.H11MO.0.A' 'THA11_HUMAN.H11MO.0.B'
 'HSF1_HUMAN.H11MO.0.A' 'HSF4_HUMAN.H11MO.0.D']
Int64Index([3864, 3087, 5287, 5625, 3

## plot a specific TF 

In [254]:
def plotallcomparisonsthatare_D21vsT21_TFlist(TFlist=["HSF1", "HSF2", "HXB2", "HSF4"]):
    height=500
    width=500
    print(TFlist)
    comparelist= allTFsallcomparisonsdf[allTFsallcomparisonsdf["same_person"]!=True]["shortname"].unique()
    print (comparelist)
    subset_sigTFsallcomparisonsdf = filterdata_on_comp_and_TFs(allTFsallcomparisonsdf=allTFsallcomparisonsdf,comparelist=comparelist,TFlist=TFlist)
    plotsortedscoreandpvalcomparison_nosize(subset_sigTFsallcomparisonsdf, height=height,width=width, category_orders=category_orders)
    

plotallcomparisonsthatare_D21vsT21_TFlist()

['HSF1', 'HSF2', 'HXB2', 'HSF4']
['ATAC_D2137_T2137' 'ATAC_D2142_T2142' 'PRO_D2137_T2137' 'PRO_D2142_T2142']

Int64Index([5766, 5039, 3540, 4289, 3307, 5399, 5320, 3944, 5400, 5249, 3297,
            3910, 3299, 3921, 5254, 5398, 5397, 3906, 3309, 4813],
           dtype='int64')


In [229]:
def violinallpvals(allTFsallcomparisonsdf):
    fig = px.violin(allTFsallcomparisonsdf, y=TFEA_pvalcolumn, x="shortname", color="same_person")
    fig.show()

In [230]:
violinallpvals(allTFsallcomparisonsdf)